In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import copy
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

BEFORE RUNNING THE NOTEBOOK: Please note that for the case of the project's final presentation, the statistics shown was unfortunately not saved as an output on the notebook, therefore due to the nature of CNN, results might be different from what was presented. The trends, however, should still be relatively similar!

Brief Overview of the Raw Data: The raw data provided in the HAPT dataset consists of time series data collected from multiple experiments. The data collected are the axial readings from the device's accelerometer and gyroscope (x, y and z axis), over a varying length of time. These readings are then processed by the curator of the dataset to produce the processed data filled with 561 attributes. However, in this notebook, instead of using the processed data, we were interested in finding out if we could create a model that could try to capture relationships within the time series data itself. For this, we looked towards CNN's. While CNN's are commonly used for problems such as image classification (spatial relationships), CNNs were known to be capable of capturing temporal relationships as well. In addition, since the 6 raw attributes formed the basis of the formulation of 561 unique attributes, we believed that there were relationships between the raw attributes that could be captured by CNNs as well. We begin by pre-processing the data.

We first extract out datasets from the raw data.

In [ ]:
def get_label_dataset(path):
    X = pd.read_csv(path, names=['exp_id', 'user_id', 'label', 'start_row', 'end_row'], sep=' ')
    return X

subject_train = get_label_dataset("/kaggle/input/hapt-data/HAPT Data Set/RawData/labels.txt")
difference = []
for i in range(len(subject_train)):
    difference.append(subject_train.iloc[i]['end_row'] - subject_train.iloc[i]['start_row'])
subject_train['difference'] = difference
common_number = subject_train.min()['difference']
subject_train

In [ ]:

def get_dataset(path):
    X = pd.read_csv(path)
    return X

base_raw = get_dataset("/kaggle/input/hatp-raw/labeled_raw_data.csv")
counter = 0
starting_label = base_raw.iloc[0]['activity']
drop_list = []
for i in range(len(base_raw)):
    if (base_raw.iloc[i]['activity'] != starting_label):
        counter = 0
        starting_label = base_raw.iloc[i]['activity']
    if (counter >= common_number):
        drop_list.append(i)
    counter += 1


In [ ]:
equalized_raw = base_raw.drop(drop_list)
labels = []
start = equalized_raw.iloc[0]['activity']
labels.append(starting_label)
for i in range(len(equalized_raw)):
    if (equalized_raw.iloc[i]['activity'] != start):
        start = equalized_raw.iloc[i]['activity']
        labels.append(start)


Now that we have retrieved the relevant indices of each instance, we ran into our first problem. Different instances had different numbers of rows of data, i.e. 
the instance was measured for different periods of time. This was problematic as CNN's need to have a fixed input size. The solution we adopted in this case was to find the instance with the smallest number of rows, which we will call n, and then extract the first n rows from every instance and make that as our dataset. This approach was simple to implement, but it proved to be a suboptimal way of processing the raw data, which we will elaborate on later.

In [ ]:
no_label_data = equalized_raw.drop('activity', 1)
n = common_number
dataset_split = [no_label_data[i:i+n].transpose() for i in range(0,equalized_raw.shape[0],n)]
f = open('CNN_train.csv', 'a')
for df in dataset_split:
    df.to_csv(f)
f.close()

In [ ]:
dict = {'labels': labels}
label_df = pd.DataFrame(dict)
label_df.to_csv('train_labels.csv')


We split the data into training and test sets here, setting 20% of the data to be in our test set. 

In [ ]:

data = np.array(dataset_split)
labels = label_df.to_numpy()

data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.20, random_state=42)
data_train_expanded_list = []
data_test_expanded_list = []
for i in range(len(data_train)):
    data_train_expanded_list.append(np.expand_dims(data_train[i], axis=2))
    
for i in range(len(data_test)):
    data_test_expanded_list.append(np.expand_dims(data_test[i], axis=2))
    
for i in range(len(labels_train)):
    labels_train[i] = labels_train[i] - 1
    
for i in range(len(labels_test)):
    labels_test[i] = labels_test[i] - 1
    
data_train_expanded = np.array(data_train_expanded_list)
data_test_expanded = np.array(data_test_expanded_list)



We begin creating our CNN model here, with parameters such as padding, number of filters as well as weights being set arbitrarily. In the introduction, we mentioned that we wanted to see if we could capture relationships across time / the raw attributes, which means that we need to take a look at filter sizes (height x width), as these dimensions allow us to capture said relationships, if they exist. So, let us explore how kernel dimensionality affects the performance of the model!  

As a side note, we will be using the F1-scores as our main metric of performance. This is because our dataset is heavily imbalanced, thus accuracy would not be the best metric to determine if our model is performing well.

In [ ]:
for i in range(1, 7):
    model1 = Sequential()
    model1.add(Conv2D(32, kernel_size=i, activation='relu', input_shape=(6,73,1), padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(Flatten())
    model1.add(Dense(64, activation='relu'))
    model1.add(Dense(12, activation='softmax'))
    model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model1.fit(data_train_expanded, labels_train, validation_split=0.2, epochs=20, verbose=0)
    y_pred = model1.predict(data_test_expanded)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(labels_test, y_pred_bool, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))
    
    
    

As we can see, there is a generally increasing trend in the performance as the height and width of the kernel increases, in particular the avg of the f1 scores. This suggests that there could be patterns to be observed either across time or across the 6 attributes. Let us examine this further.

In [ ]:
model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(6, 1), activation='relu', input_shape=(6,73,1), padding='same'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dense(12, activation='softmax'))
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.fit(data_train_expanded, labels_train, validation_split=0.2, epochs=20, verbose=0)
y_pred2 = model2.predict(data_test_expanded)
y_pred_bool2 = np.argmax(y_pred2, axis=1)
print(classification_report(labels_test, y_pred_bool2, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))

model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(1, 12), activation='relu', input_shape=(6,73,1), padding='same'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model3.add(Flatten())
model3.add(Dense(64, activation='relu'))
model3.add(Dense(12, activation='softmax'))
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.fit(data_train_expanded, labels_train, validation_split=0.2, epochs=20, verbose=0)
y_pred3 = model3.predict(data_test_expanded)
y_pred_bool3 = np.argmax(y_pred3, axis=1)
print(classification_report(labels_test, y_pred_bool3, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))




From the stats above, despite not having a significant difference, there seems to be a stronger link across the 6 attributes than across time. Nevertheless, the lack of a significant difference suggests that both play a somewhat similar significance in the performance of the model. Let's try to confirm this by changing the rows and columns a little. In the cell below, we will be swapping rows 2, 3 with rows 4, 5 respectively.

In [ ]:
train_row_swapped = copy.deepcopy(data_train)
test_row_swapped = copy.deepcopy(data_test)

for i in range(len(train_row_swapped)):
    train_row_swapped[i][[1, 3]] = train_row_swapped[i][[3, 1]]
    train_row_swapped[i][[2, 5]] = train_row_swapped[i][[5, 2]]

for i in range(len(test_row_swapped)):
    test_row_swapped[i][[1, 3]] = test_row_swapped[i][[3, 1]]
    test_row_swapped[i][[2, 5]] = test_row_swapped[i][[5, 2]]

train_row_swapped_expanded_list = []
test_row_swapped_expanded_list = []
for i in range(len(data_train)):
    train_row_swapped_expanded_list.append(np.expand_dims(train_row_swapped[i], axis=2))
    
for i in range(len(data_test)):
    test_row_swapped_expanded_list.append(np.expand_dims(test_row_swapped[i], axis=2))
    
    
train_rse = np.array(train_row_swapped_expanded_list)
test_rse = np.array(test_row_swapped_expanded_list)


for i in range(1, 7):
    model1 = Sequential()
    model1.add(Conv2D(32, kernel_size=i, activation='relu', input_shape=(6,73,1), padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(Flatten())
    model1.add(Dense(64, activation='relu'))
    model1.add(Dense(12, activation='softmax'))
    model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model1.fit(train_rse, labels_train, validation_split=0.2, epochs=20, verbose=0)
    y_pred = model1.predict(test_rse)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(labels_test, y_pred_bool, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))






From the statistics above, it seems that attribute order did play a significant difference in the performance of the model. It is interesting to note that while the performance for the 6-height kernel decreased, the performances of the other kernels improved, specifically kernels of size 2, 3 and 5. This could be explained by how the rows were arranged to better allow for smaller kernels to capture important relationships between the attributes.(For example, one of the attributes selected when we applied LDA on the processed was tBodyAccJerk-Min-3, which involved using the z-axis data (rows 2 and 5) from the original training data.) Now let's try randomizing the columns in the training data (time).

In [ ]:
train_col_swapped = copy.deepcopy(data_train)

for i in range(len(train_col_swapped)):
    train_col_swapped[i] = train_col_swapped[i][:, np.random.permutation(train_col_swapped[i].shape[1])]

train_col_swapped_expanded_list = []

for i in range(len(data_train)):
    train_col_swapped_expanded_list.append(np.expand_dims(train_col_swapped[i], axis=2))
    
train_cse = np.array(train_col_swapped_expanded_list)


for i in range(1, 7):
    model1 = Sequential()
    model1.add(Conv2D(32, kernel_size=i, activation='relu', input_shape=(6,73,1), padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(Flatten())
    model1.add(Dense(64, activation='relu'))
    model1.add(Dense(12, activation='softmax'))
    model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model1.fit(train_cse, labels_train, validation_split=0.2, epochs=20, verbose=0)
    y_pred = model1.predict(data_test_expanded)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(labels_test, y_pred_bool, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))



As expected, the performances of the models decreases by a significant amount. This proves that there is a strong link between the order of the columns (time) and the overall performance, which makes sense considering our raw data is a time-series data. Now, lets try to improve on the performance of our CNN model by including drop out.

In [ ]:
for i in range(1, 7):
    model1 = Sequential()
    model1.add(Conv2D(32, kernel_size=i, activation='relu', input_shape=(6,73,1), padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Dropout(0.2))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Dropout(0.2))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(Flatten())
    model1.add(Dense(64, activation='relu'))
    model1.add(Dense(12, activation='softmax'))
    model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model1.fit(data_train_expanded, labels_train, validation_split=0.2, epochs=20, verbose=0)
    y_pred = model1.predict(data_test_expanded)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(labels_test, y_pred_bool, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))

As we can see, there isn't a significant change in overall performances of the models. This suggests that there is not much overfitting that was happening in our original models. We can confirm this by examining the losses during the training. The cell below compares the loss without dropout.

In [ ]:
model2 = Sequential()
model2.add(Conv2D(32, kernel_size=6, activation='relu', input_shape=(6,73,1), padding='same'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dense(12, activation='softmax'))
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.fit(data_train_expanded, labels_train, validation_split=0.2, epochs=20, verbose=0)
history2 = model2.history.history
train_loss2 = history2['loss']
val_loss2 = history2['val_loss']

plt.plot(train_loss2, label="Training Loss")
plt.plot(val_loss2, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)

The cell below compares the loss with dropout. As we can see there is not much differences in the losses.

In [ ]:
model1 = Sequential()
model1.add(Conv2D(32, kernel_size=i, activation='relu', input_shape=(6,73,1), padding='same'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Dropout(0.2))
model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Dropout(0.2))
model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dense(12, activation='softmax'))
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.fit(data_train_expanded, labels_train, validation_split=0.2, epochs=20, verbose=0)
history1 = model1.history.history
train_loss1 = history1['loss']
val_loss1 = history1['val_loss']

plt.plot(train_loss1, label="Training Loss")
plt.plot(val_loss1, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)

Another thing we could do to improve the performance of our models is to target our training data. There is a clear imbalance in the data types, especially towards labels 1 - 6. We could make our training data more balanced by using smote!

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
smote = SMOTE()
nsamples, nx, ny = data_train.shape
d2_data_train = data_train.reshape((nsamples,nx*ny))
x_train_smoted, balanced_train_labels = smote.fit_resample(d2_data_train, labels_train)
newsamples, nxy = x_train_smoted.shape
balanced_train = np.reshape(x_train_smoted, (newsamples, 6, 73))
balanced_train_expanded_list = []
for i in range(len(balanced_train)):
    balanced_train_expanded_list.append(np.expand_dims(balanced_train[i], axis=2))
balanced_train_expanded = np.array(balanced_train_expanded_list)


Now let's examine the performances of our models again!

In [ ]:
for i in range(1, 7):
    model1 = Sequential()
    model1.add(Conv2D(32, kernel_size=i, activation='relu', input_shape=(6,73,1), padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Dropout(0.2))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Dropout(0.2))
    model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model1.add(Flatten())
    model1.add(Dense(64, activation='relu'))
    model1.add(Dense(12, activation='softmax'))
    model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model1.fit(balanced_train_expanded, balanced_train_labels, validation_split=0.2, epochs=20, verbose=0)
    y_pred = model1.predict(data_test_expanded)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(labels_test, y_pred_bool, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))

As we can see, there is a general increase in performance across all models! Now, let us examine a few microscopic details. From all the examples we have seen, there are a few labels that are consistently performing poorly in terms of the f1 score. These labels are: 1, 10 and 12, which are walking, lie-to-sit and lie-to-stand. For these instances, let us see what is the most common prediction, using our height-6 kernel as our test model as well as balanced training data.

In [ ]:
from sklearn import metrics

model1 = Sequential()
model1.add(Conv2D(32, kernel_size=6, activation='relu', input_shape=(6,73,1), padding='same'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Dropout(0.2))
model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Dropout(0.2))
model1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dense(12, activation='softmax'))
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.fit(balanced_train_expanded, balanced_train_labels, validation_split=0.2, epochs=20, verbose=1)
y_pred = model1.predict(data_test_expanded)
y_pred_bool = np.argmax(y_pred, axis=1)

one_dict = {}
ten_dict = {}
twelve_dict = {}

for i in range(len(labels_test)):
    if (labels_test[i] == 0):
        prediction = y_pred_bool[i] + 1
        if prediction in one_dict:
            one_dict[prediction] = one_dict[prediction] + 1
        else:
            one_dict[prediction] = 1
    elif (labels_test[i] == 9):
        prediction = y_pred_bool[i] + 1
        if prediction in ten_dict:
            ten_dict[prediction] = ten_dict[prediction] + 1
        else:
            ten_dict[prediction] = 1
    elif (labels_test[i] == 11):
        prediction = y_pred_bool[i] + 1
        if prediction in twelve_dict:
            twelve_dict[prediction] = twelve_dict[prediction] + 1
        else:
            twelve_dict[prediction] = 1

cnf_matrix = metrics.confusion_matrix(labels_test, y_pred_bool)
cnf_matrix
        

The cell below graphs the predictions, and their counts, of label 1 instances in our test data. As we can see, it is having trouble distinguishing 1 with instances 3 and 2, which are largely similar to each other. It is at least able to easily distinguish 1 from the other labels, which are vastly different actions.

In [ ]:
predictions = list(one_dict.keys())
counts = list(one_dict.values())

plt.bar(range(len(one_dict)), counts, tick_label=predictions)
plt.show()

With the label 10 instances, our model seems to heavy incline towards label 12, with a few instances of label 6. These actions are somewhat similar to each other, as they all involve lying down at the beginning, which shows that our model is at least able to predict actions that are similar to each other once again.

In [ ]:
predictions = list(ten_dict.keys())
counts = list(ten_dict.values())

plt.bar(range(len(ten_dict)), counts, tick_label=predictions)
plt.show()

For label 12 instances, the model is having trouble distinguishing between labels 10 and labels 12, which as mentioned before, is largely similar in the way that they begin by lying down.

In [ ]:
predictions = list(twelve_dict.keys())
counts = list(twelve_dict.values())

plt.bar(range(len(twelve_dict)), counts, tick_label=predictions)
plt.show()

Based on what we have seen, we can see that the model struggles to identify actions that begin similarly to other actions. For example, labels 1 with 2 and 3, labels 10 with 12 and 6. There is a clear explanation for this observation! This is because due to CNN's having a fixed input shape, and that the raw data has a varying number of time-steps recorded for each instance, we had to cut down on the number of columns (time) used per instance to the minimum number of columns observed in a single instance. This means that for many instances, only information of the early stages of the  are captured (for example, when we take a look at labels 10 and 12, cutting down of columns likely meant that information on the actions taken after lying down has been largely removed).

Therefore, to fix this issue, we need to pre-process our data better by including columns that are located further down each instance in order to capture the information lost. However, for the purposes of this project, due to how tedious it is to pre-process our raw data this way (varying number of rows per instance make it difficult to decide which columns to include per instance), we did not explore this possibility. Nevertheless, we theorize that doing so will improve the performance of CNN on our chosen dataset. 

Overall, despite the limitation mentioned above, CNN still performed relatively well, achieving relatively high f1 scores, which demonstrates the ability of CNNs to capture relationships across time. 